In [1]:
import numpy as np
from numba.typed import List as NumbaList
from BPnumba.NumAG import InstancePob,TrainAG
from BPmodule.BPGen  import PROBLEM2,PROBLEM3,PROBLEM4,PROBLEM5
from BPnumba.NumFun import CreatePoblation
from numba import njit,prange,objmode
from typing import List
from BPnumba.DFFAnumba import DFFtrain
import time

ImportError: cannot import name 'ind_type' from partially initialized module 'BPnumba.NumAG' (most likely due to a circular import) (c:\Users\nicoo\Desktop\BPP3Dpython\BPnumba\NumAG.py)

In [2]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]

In [3]:
gamma=0.0004

In [4]:
contenedor =np.asanyarray( P2A2[0],dtype=np.int64)
datos =np.asanyarray(P2A2[1],dtype=np.int64)
n = len(datos)
IniPob = CreatePoblation(1, datos)
fireflyPob = InstancePob(NumbaList(IniPob),NumbaList(datos),NumbaList(contenedor))
ind  =DFFtrain(1,NumbaList(fireflyPob),gamma,datos,contenedor)
contenedor = np.asanyarray(P2A1[0],dtype=np.int64)
datos =np.asanyarray(P2A1[1],dtype=np.int64)
initial = CreatePoblation(1,datos)
start_time = time.time()
Pob =InstancePob(NumbaList(initial),datos,contenedor)
TrainAG(2,NumbaList(Pob),0.85,0.75,0.2,NumbaList(datos),NumbaList(contenedor))
print("--- %s seconds ---" % (time.time() - start_time))


c:\Users\nicoo\Anaconda3\lib\site-packages\numba\typed\typedlist.py:83: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'item' of function 'impl_append.<locals>.impl'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\Anaconda3\lib\site-packages\numba\typed\listobject.py", line 599:

    def impl(l, item):
    ^

  l.append(item)
c:\Users\nicoo\Anaconda3\lib\site-packages\numba\core\ir_utils.py:2139: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'item' of function '_append'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\Anaconda3\lib\site-packages\numba\typed\typedlist.py", line 82:
@nji

--- 16.40111494064331 seconds ---


In [5]:
@njit(parallel=True)
def TestAlg(maxTest, maxItr,pop,datos, contenedor):
    initial = CreatePoblation(pop,datos)
    print('Inicia DFF')
    for _ in prange(maxTest):
        with objmode(time1='f8'):
            time1 = time.perf_counter()
        Pob = InstancePob(NumbaList(initial.copy()),datos,contenedor)
        best =DFFtrain(maxItr,NumbaList(Pob),0.004,NumbaList(datos),NumbaList(contenedor))
        print(best.fi,len(datos)-best.load)
        with objmode():
            print('time: {}'.format(time.perf_counter() - time1))
    print('Inicia GA')
    for _ in prange(maxTest):
        with objmode(time1='f8'):
            time1 = time.perf_counter()
        Pob = InstancePob(NumbaList(initial.copy()),datos,contenedor)
        best =TrainAG(maxItr,NumbaList(Pob),0.85,0.75,0.2,NumbaList(datos),NumbaList(contenedor))
        print(best.fi,len(datos)-best.load)
        with objmode():
            print('time: {}'.format(time.perf_counter() - time1))
    print('')

In [6]:
contenedor =np.asanyarray( P2A2[0],dtype=np.int64)
datos =np.asanyarray(P2A2[1],dtype=np.int64)
TestAlg(5,500,50,datos,contenedor)

Inicia DFF
1.0 0
time: 120.844815
1.0 0
time: 121.40723720000001
1.0 0
time: 122.10542410000001
1.0 0
time: 122.24582899999999
1.0 0
time: 124.1021784
Inicia GA
0.9928 1
time: 6.1241986
0.9928 1
time: 6.4563402
0.9928 1
time: 8.599726400000009
0.9928 1
time: 9.600072600000004
0.99916 1
time: 14.354832600000009



In [7]:
contenedor =np.asanyarray( P2A1[0],dtype=np.int64)
datos =np.asanyarray(P2A1[1],dtype=np.int64)
TestAlg(5,500,50,datos,contenedor)

Inicia DFF
0.998144 1
time: 534.2750293
1.0 0
time: 535.1940917
1.0 0
time: 536.1779841
1.0 0
time: 538.5974747
1.0 0
time: 540.4451273000001
Inicia GA
0.995392 1
time: 93.39071430000001
0.998144 1
time: 102.63342599999999
1.0 0
time: 104.64341750000006
0.994896 1
time: 109.6629279
0.999472 1
time: 111.42524490000005



In [8]:
contenedor =np.asanyarray( P3A1[0],dtype=np.int64)
datos =np.asanyarray(P3A1[1],dtype=np.int64)
TestAlg(1,500,50,datos,contenedor)

Inicia DFF
0.891892 2
time: 98.75465670000006
Inicia GA
0.987697 2
time: 541.0805544



In [9]:
contenedor =np.asanyarray( P3A2[0],dtype=np.int64)
datos =np.asanyarray(P3A2[1],dtype=np.int64)
TestAlg(1,500,50,datos,contenedor)

Inicia DFF
0.96216 6
time: 477.2970568999999
Inicia GA
0.922 1
time: 536.9404608

